In [35]:
# Dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st
import sqlite3
import csv
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
dfSurvivals = pd.read_excel("dataset/titanic3.xls", index_col=None)
dfSurvivals.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
#dfSurvivals.drop(["cabin", "home.dest", "ticket"], axis=1, inplace=True)

In [4]:
#dfSurvivals["fare"].fillna(value = dfSurvivals["fare"].mean(), inplace=True)
#dfSurvivals["embarked"].fillna(dfSurvivals["embarked"].value_counts().idxmax(), inplace=True)

In [12]:
# Split the column name into Last name, Title, First Name
#dfSurvivals[['Last Name', 'Title First Name']] = dfSurvivals["name"].str.split(',', expand=True)
#dfSurvivals[['Title', 'First Name']] = dfSurvivals["Title First Name"].str.split('.', n=1, expand=True)
#dfSurvivals = dfSurvivals.drop(columns = ['name', 'Title First Name'], axis = 1)
#dfSurvivals['Passenger Id'] = dfSurvivals.index + 1

In [17]:
dfSurvivals["boat"].fillna('DNS', inplace=True)
dfSurvivals["body"] = np.where(dfSurvivals["survived"] == 1, dfSurvivals["body"].fillna('S'), dfSurvivals["body"])
dfSurvivals["body"].fillna('BNR', inplace=True)

In [25]:
# Keep and reorder columns we're interested in
dfSurvivals=dfSurvivals[["Passenger Id", "Title", "First Name", "Last Name", "sex", "age", "sibsp", "parch", "pclass",
                             "fare", "embarked", "boat", "body", "survived"]]

In [10]:
dfSurvivals.groupby(["Title", "sex", "pclass"])["age"].median()

Title          sex     pclass
 Capt          male    1         70.0
 Col           male    1         54.5
 Don           male    1         40.0
 Dona          female  1         39.0
 Dr            female  1         49.0
               male    1         47.0
                       2         38.5
 Jonkheer      male    1         38.0
 Lady          female  1         48.0
 Major         male    1         48.5
 Master        male    1          6.0
                       2          2.0
                       3          6.0
 Miss          female  1         30.0
                       2         20.0
                       3         18.0
 Mlle          female  1         24.0
 Mme           female  1         24.0
 Mr            male    1         41.5
                       2         30.0
                       3         26.0
 Mrs           female  1         45.0
                       2         30.5
                       3         31.0
 Ms            female  2         28.0
                    

In [22]:
#dfSurvivals.dropna(subset=["age"], inplace=True)

In [23]:
dfSurvivals["age"] = dfSurvivals.groupby(["Title", "sex", "pclass"])["age"].apply(lambda x: x.fillna(x.median()))
print(f"Number of NA values: \n{dfSurvivals.isna().sum()}")

Number of NA values: 
Passenger Id    0
Title           0
First Name      0
Last Name       0
sex             0
age             0
sibsp           0
parch           0
pclass          0
fare            0
embarked        0
boat            0
body            0
survived        0
dtype: int64


C:\Users\juank\AppData\Local\Temp\ipykernel_3804\30495221.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dfSurvivals["age"] = dfSurvivals.groupby(["Title", "sex", "pclass"])["age"].apply(lambda x: x.fillna(x.median()))


In [31]:
# Rename columns
dfSurvivals = dfSurvivals.rename(
    columns={"sex": "Sex",
            "age": "Age",
            "sibsp": "Sibling/Spouse Abroad",
            "parch": "Parent/Children Abroad",
            "ticket": "Ticket Number",
            "pclass": "Passenger Class",
            "fare": "Fare",
            "embarked": "Embarkation Port",
            "boat": "Survival Boat",
            "body": "Body Number",
            "home.dest": "Home/Destination",
            "survived": "Survived"})

In [32]:
#dfSurvivals.drop(["Title","First Name","Last Name"], axis=1, inplace=True)

In [37]:
result = OneHotEncoder().fit_transform(dfSurvivals["Sex"].values.reshape(-1, 1)).toarray()
dfSurvivals[["Female", "Male"]] = pd.DataFrame(result, index = dfSurvivals.index)
dfSurvivals.drop(["Sex"], axis=1, inplace=True)

In [39]:
labelencoder_X = LabelEncoder()

dfSurvivals["Embarkation Port"] = labelencoder_X.fit_transform(dfSurvivals["Embarkation Port"])

In [40]:
dfSurvivals.head()

,Passenger Id,Age,Sibling/Spouse Abroad,Parent/Children Abroad,Passenger Class,Fare,Embarkation Port,Survival Boat,Body Number,Survived,Female,Male
0,1,29.0000,0,0,1,211.3375,2,2,S,1,1.0,0.0
1,2,0.9167,1,2,1,151.5500,2,11,S,1,0.0,1.0
2,3,2.0000,1,2,1,151.5500,2,DNS,BNR,0,1.0,0.0
3,4,30.0000,1,2,1,151.5500,2,DNS,135.0,0,0.0,1.0
4,5,25.0000,1,2,1,151.5500,2,DNS,BNR,0,1.0,0.0


In [41]:
dfSurvivals.to_csv('ML_Cleaned_DS', index=False)